In [ ]:
# Imports
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import matplotlib.pyplot as plt
import operator
import json
import os

print("All imports successful")

In [ ]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("API key loaded")

In [ ]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"LLM initialized: {llm.model_name}")

---

## Exercise 1: Adaptive Reflection with Quality Metrics

Improve Reflection pattern with numerical quality scoring using Pydantic models.

In [ ]:
# Pydantic model for quality scoring
class QualityScore(BaseModel):
    """Quality scores for draft evaluation."""
    clarity: int = Field(description="How clear and understandable is the response? (1-5)", ge=1, le=5)
    completeness: int = Field(description="How complete and thorough is the response? (1-5)", ge=1, le=5)
    accuracy: int = Field(description="How accurate and correct is the response? (1-5)", ge=1, le=5)
    feedback: str = Field(description="Specific feedback on what needs improvement")
    
    def average_score(self) -> float:
        return (self.clarity + self.completeness + self.accuracy) / 3
    
    def needs_refinement(self) -> bool:
        """Check if any score is below 4."""
        return min(self.clarity, self.completeness, self.accuracy) < 4

print("✅ QualityScore model defined")

In [ ]:
# State for adaptive reflection
class AdaptiveReflectionState(TypedDict):
    """State for reflection with quality metrics."""
    task: str
    draft: str
    scores: Annotated[list[QualityScore], operator.add]
    iterations: int
    final_output: str

MAX_ITERATIONS = 3

print("AdaptiveReflectionState defined")

In [ ]:
# Bind structured output to LLM
llm_with_scoring = llm.with_structured_output(QualityScore)

# Node 1: Generator
def adaptive_generator(state: AdaptiveReflectionState) -> dict:
    """Generate or refine based on quality scores."""
    if state["iterations"] == 0:
        prompt = f"""Create a response for this task:

Task: {state['task']}

Provide a clear, complete, and accurate answer."""
        print("\nGenerating initial draft...")
    else:
        last_score = state["scores"][-1]
        prompt = f"""Improve this draft based on the quality feedback:

Task: {state['task']}

Current draft: {state['draft']}

Quality Scores:
- Clarity: {last_score.clarity}/5
- Completeness: {last_score.completeness}/5
- Accuracy: {last_score.accuracy}/5

Feedback: {last_score.feedback}

Create an improved version that addresses the feedback and improves the scores."""
        print(f"\nRefining (iteration {state['iterations']})...")
    
    response = llm.invoke([HumanMessage(content=prompt)])
    print("✓ Draft created\n")
    
    return {"draft": response.content}

# Node 2: Critic with scoring
def adaptive_critic(state: AdaptiveReflectionState) -> dict:
    """Evaluate draft and provide quality scores."""
    prompt = f"""Evaluate this response using the following criteria (score 1-5 each):

Task: {state['task']}

Response: {state['draft']}

Criteria:
1. Clarity: How clear and understandable is it?
2. Completeness: How thorough is it?
3. Accuracy: How correct and reliable is it?

For each criterion, give a score from 1-5:
- 5: Excellent
- 4: Good
- 3: Acceptable
- 2: Needs improvement
- 1: Poor

Also provide specific feedback on what needs improvement."""
    
    print("Evaluating draft...")
    score = llm_with_scoring.invoke([HumanMessage(content=prompt)])
    
    print(f" Scores: Clarity={score.clarity}, Completeness={score.completeness}, Accuracy={score.accuracy}")
    print(f" Feedback: {score.feedback[:100]}...\n")
    
    return {
        "scores": [score],
        "iterations": state["iterations"] + 1
    }

# Node 3: Finalizer
def adaptive_finalizer(state: AdaptiveReflectionState) -> dict:
    """Finalize output and show score progression."""
    print("\n Reflection complete!\n")
    print(" Score Progression:")
    for i, score in enumerate(state["scores"]):
        print(f"  Iteration {i+1}: Clarity={score.clarity}, Completeness={score.completeness}, Accuracy={score.accuracy} (Avg: {score.average_score():.2f})")
    print()
    
    return {"final_output": state["draft"]}

print(" Adaptive reflection nodes defined")

In [ ]:
# Routing based on quality scores
def should_refine(state: AdaptiveReflectionState) -> Literal["generator", "finalizer"]:
    """Decide if refinement needed based on scores."""
    if not state.get("scores"):
        return "finalizer"
    
    last_score = state["scores"][-1]
    
    # Stop if all scores >= 4
    if not last_score.needs_refinement():
        print(" All quality scores >= 4. Approved!\n")
        return "finalizer"
    
    # Stop if max iterations reached
    if state["iterations"] >= MAX_ITERATIONS:
        print(f" Max iterations ({MAX_ITERATIONS}) reached\n")
        return "finalizer"
    
    return "generator"

# Build graph
adaptive_builder = StateGraph(AdaptiveReflectionState)

adaptive_builder.add_node("generator", adaptive_generator)
adaptive_builder.add_node("critic", adaptive_critic)
adaptive_builder.add_node("finalizer", adaptive_finalizer)

adaptive_builder.add_edge(START, "generator")
adaptive_builder.add_edge("generator", "critic")
adaptive_builder.add_conditional_edges(
    "critic",
    should_refine,
    {"generator": "generator", "finalizer": "finalizer"}
)
adaptive_builder.add_edge("finalizer", END)

adaptive_reflection_agent = adaptive_builder.compile()

print(" Adaptive reflection agent created")

In [ ]:
# Visualize graph
try:
    display(Image(adaptive_reflection_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")

### Test Adaptive Reflection

In [ ]:
# Test with a task that requires refinement
result = adaptive_reflection_agent.invoke({
    "task": "Explain quantum computing to a 10-year-old in 3-4 sentences",
    "draft": "",
    "scores": [],
    "iterations": 0
})

print(f"\n{'='*70}")
print(" FINAL OUTPUT:")
print(f"{'='*70}")
print(result["final_output"])
print(f"\nTotal iterations: {result['iterations']}")
print(f"{'='*70}\n")

In [ ]:
# Visualize score improvements
def visualize_scores(scores: list[QualityScore]):
    """Plot score improvements across iterations."""
    iterations = list(range(1, len(scores) + 1))
    clarity = [s.clarity for s in scores]
    completeness = [s.completeness for s in scores]
    accuracy = [s.accuracy for s in scores]
    
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, clarity, marker='o', label='Clarity', linewidth=2)
    plt.plot(iterations, completeness, marker='s', label='Completeness', linewidth=2)
    plt.plot(iterations, accuracy, marker='^', label='Accuracy', linewidth=2)
    plt.axhline(y=4, color='green', linestyle='--', alpha=0.5, label='Target (4.0)')
    
    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.title('Quality Score Progression', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 5.5)
    plt.xticks(iterations)
    plt.tight_layout()
    plt.show()

visualize_scores(result["scores"])



## Exercise 2: Plan-Execute + Reflection Hybrid



In [ ]:
# Hybrid state combining both patterns
class HybridState(TypedDict):
    """Combined Plan-Execute and Reflection state."""
    # Plan-Execute components
    input: str
    plan: list[str]
    current_step: int
    results: Annotated[list[str], operator.add]
    
    # Reflection components
    draft: str
    critique: str
    reflection_iterations: int
    
    # Final output
    final_output: str

MAX_REFLECTIONS = 2

print(" HybridState defined")

In [ ]:
# Node 1: Planner
def hybrid_planner(state: HybridState) -> dict:
    """Create step-by-step plan."""
    prompt = f"""Create a step-by-step plan for this task:

Task: {state['input']}

Return a numbered list of 3-5 concrete steps."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Parse steps
    lines = response.content.split('\n')
    steps = [line.strip() for line in lines if line.strip() and any(char.isdigit() for char in line[:3])]
    
    print(f"\n{'='*70}")
    print(" PLAN CREATED:")
    print(f"{'='*70}")
    for step in steps:
        print(f"  {step}")
    print(f"{'='*70}\n")
    
    return {"plan": steps, "current_step": 0, "results": []}

# Node 2: Executor
def hybrid_executor(state: HybridState) -> dict:
    """Execute current step."""
    if state["current_step"] >= len(state["plan"]):
        return {}
    
    current_step = state["plan"][state["current_step"]]
    
    print(f"\n EXECUTING STEP {state['current_step'] + 1}:")
    print(f"   {current_step}")
    
    # Execute step
    prompt = f"""Execute this step:

Overall task: {state['input']}

Previous results: {state.get('results', [])}

Current step: {current_step}

Provide a detailed result for this step."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    result = f"Step {state['current_step'] + 1}: {response.content}"
    
    print(f"   ✓ Completed\n")
    
    return {
        "results": [result],
        "current_step": state["current_step"] + 1
    }

# Node 3: Generator (creates initial draft from results)
def hybrid_generator(state: HybridState) -> dict:
    """Generate initial output from execution results."""
    if state["reflection_iterations"] == 0:
        # First generation from execution results
        prompt = f"""Synthesize these step results into a cohesive response:

Original task: {state['input']}

Results from each step:
{chr(10).join(state['results'])}

Create a clear, complete response that addresses the original task."""
        print("\n GENERATING INITIAL DRAFT FROM RESULTS...")
    else:
        # Refinement based on critique
        prompt = f"""Improve this draft based on the critique:

Task: {state['input']}

Current draft: {state['draft']}

Critique: {state['critique']}

Create an improved version."""
        print(f"\n REFINING DRAFT (iteration {state['reflection_iterations']})...")
    
    response = llm.invoke([HumanMessage(content=prompt)])
    print("   ✓ Draft created\n")
    
    return {"draft": response.content}

# Node 4: Critic
def hybrid_critic(state: HybridState) -> dict:
    """Critique the complete output."""
    prompt = f"""Evaluate this response:

Task: {state['input']}

Response: {state['draft']}

Provide constructive critique. What could be improved?
If it's excellent, start with "APPROVED:".
Otherwise, provide specific improvements needed."""
    
    print(" CRITIQUING DRAFT...")
    response = llm.invoke([HumanMessage(content=prompt)])
    critique = response.content
    
    print(f"   Critique: {critique[:150]}...\n")
    
    return {
        "critique": critique,
        "reflection_iterations": state["reflection_iterations"] + 1
    }

# Node 5: Finalizer
def hybrid_finalizer(state: HybridState) -> dict:
    """Finalize output."""
    print("\n HYBRID AGENT COMPLETE!\n")
    print(f"Total reflection iterations: {state['reflection_iterations']}\n")
    return {"final_output": state["draft"]}

print(" Hybrid nodes defined")

In [ ]:
# Routing functions
def should_continue_execution(state: HybridState) -> Literal["executor", "generator"]:
    """Check if more steps to execute."""
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "generator"

def should_reflect_again(state: HybridState) -> Literal["generator", "finalizer"]:
    """Check if refinement needed."""
    # Stop if approved
    if "APPROVED" in state.get("critique", "").upper():
        print(" Draft approved!\n")
        return "finalizer"
    
    # Stop if max iterations
    if state["reflection_iterations"] >= MAX_REFLECTIONS:
        print(f"    Max reflection iterations ({MAX_REFLECTIONS}) reached\n")
        return "finalizer"
    
    return "generator"

# Build hybrid graph
hybrid_builder = StateGraph(HybridState)

hybrid_builder.add_node("planner", hybrid_planner)
hybrid_builder.add_node("executor", hybrid_executor)
hybrid_builder.add_node("generator", hybrid_generator)
hybrid_builder.add_node("critic", hybrid_critic)
hybrid_builder.add_node("finalizer", hybrid_finalizer)

# Plan-Execute flow
hybrid_builder.add_edge(START, "planner")
hybrid_builder.add_edge("planner", "executor")
hybrid_builder.add_conditional_edges(
    "executor",
    should_continue_execution,
    {"executor": "executor", "generator": "generator"}
)

# Reflection flow
hybrid_builder.add_edge("generator", "critic")
hybrid_builder.add_conditional_edges(
    "critic",
    should_reflect_again,
    {"generator": "generator", "finalizer": "finalizer"}
)

hybrid_builder.add_edge("finalizer", END)

hybrid_agent = hybrid_builder.compile()

print("Hybrid agent created")

In [ ]:
# Visualize graph
try:
    display(Image(hybrid_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")

### Test Hybrid Agent

In [ ]:
# Test with the specified scenario
result = hybrid_agent.invoke({
    "input": "Research the benefits of Python programming, create a summary, and make it beginner-friendly",
    "plan": [],
    "current_step": 0,
    "results": [],
    "draft": "",
    "critique": "",
    "reflection_iterations": 0
})

print(f"\n{'='*70}")
print("FINAL OUTPUT:")
print(f"{'='*70}")
print(result["final_output"])
print(f"{'='*70}\n")